#### Stack Pricing Logic

##### Stack Pricing in Power Price Forecasting
In a stack-based price model, the idea is:
- Estimate demand at a given time (e.g., 7 PM tomorrow).
- Simulate the generation stack: order generators by marginal cost.
- Dispatch generation up the stack until demand is met.
- The last unit (marginal generator) sets the price.

👉 It mimics how market-clearing price is formed.

🧱 Basic Components of the Model
- Demand forecast
- Available generation units (or generation types like gas, coal, wind, etc.)
- Marginal cost of each type
- Generation capacity & availability (e.g. outages, wind forecast)
- Regulations, carbon costs, fuel prices (optional but realistic)

In [1]:
import pandas as pd

# Sample supply stack
stack = pd.DataFrame({
    'generator': ['Wind', 'Solar', 'Nuclear', 'Gas', 'Oil'],
    'cost': [0, 5, 10, 50, 100],          # €/MWh
    'available_capacity': [20, 10, 15, 25, 10]  # MW
}).sort_values('cost')
# Simulate dispatch
stack['cumulative_capacity'] = stack['available_capacity'].cumsum()
stack

,generator,cost,available_capacity,cumulative_capacity
0,Wind,0,20,20
1,Solar,5,10,30
2,Nuclear,10,15,45
3,Gas,50,25,70
4,Oil,100,10,80


In [2]:
# Forecasted demand (e.g., for 6pm tomorrow)
demand = 55
# Find the marginal generator (first row where cumulative_capacity ≥ demand)
marginal = stack[stack['cumulative_capacity'] >= demand].iloc[0]
marginal

generator              Gas
cost                    50
available_capacity      25
cumulative_capacity     70
Name: 3, dtype: object

In [4]:
market_price = marginal['cost']
print(f"Forecasted Market Price: {market_price} £/MWh")

Forecasted Market Price: 50 £/MWh


📈 Forecasting Use Case
In practice, you'd:
- Run this logic hourly or half-hourly using forecasted demand.
- Use wind/solar forecasts to limit available capacity.
- Account for dynamic fuel costs (e.g., gas, CO₂).
- Layer in ML/Statistical models on top to learn patterns from historical prices + stack signals.

🧠 Example Enhancements
- Add reserve margins, curtailments, or interconnectors.
- Include bids/offers from real market data (e.g., from Elexon in the UK).
- Combine with time series models (ARIMA, LSTM) or XGBoost using stack features.

- Look for within day demand curve, generation availability, solar, and wind forecasat
**Demand data**
- Explore how is the embedded wind and solar data can be integrated into the demand curve?
- How is the forecasted peak demand to be included in half-hourly demand curve?

**Supply/generation data**
Within day latest forecasted generation availability and generation data

**SNSP forecast**
- System non-synchronous penetration: It is the percentage of how much generation or imports that will be on the system that are not synchronised with frequency 
- 




- Forecasted prices input: (https://flex-data-docs.azurewebsites.net/enbm.forecastdapricesinputs.html)
- Source for generation https://transparency.entsoe.eu/dashboard/show


In [1]:
from entsoe import EntsoePandasClient
import pandas as pd
# import matplotlib.pyplot as pltimport 

In [2]:
# Step 1: Set up the client with your API key
client = EntsoePandasClient(api_key='YOUR_API_KEY_HERE')

# Step 2: Define parameters
country_code = 'GB'  # Great Britain
start = pd.Timestamp('2025-04-18', tz='Europe/London')
end = pd.Timestamp('2025-04-20', tz='Europe/London')  # 2 days of data

# Step 3: Query Day-Ahead Prices
prices = client.query_day_ahead_prices(country_code, start=start, end=end)

# Step 4: Display the result
print(prices.head())

HTTPError: 401 Client Error: Unauthorized for url: https://web-api.tp.entsoe.eu/api?documentType=A44&in_Domain=10YGB----------A&out_Domain=10YGB----------A&offset=0&securityToken=YOUR_API_KEY_HERE&periodStart=202504162300&periodEnd=202504202300

In [4]:
help(EntsoePandasClient)

Help on class EntsoePandasClient in module entsoe.entsoe:

class EntsoePandasClient(EntsoeRawClient)
 |  EntsoePandasClient(api_key: str, session: Optional[requests.sessions.Session] = None, retry_count: int = 1, retry_delay: int = 0, proxies: Optional[Dict] = None, timeout: Optional[int] = None)
 |  
 |  Method resolution order:
 |      EntsoePandasClient
 |      EntsoeRawClient
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  query_activated_balancing_energy(self, country_code: Union[entsoe.mappings.Area, str], start: pandas._libs.tslibs.timestamps.Timestamp, end: pandas._libs.tslibs.timestamps.Timestamp, business_type: str, psr_type: Optional[str] = None) -> pandas.core.frame.DataFrame
 |      Activated Balancing Energy [17.1.E]
 |      Parameters
 |      ----------
 |      country_code : Area|str
 |      start : pd.Timestamp
 |      end : pd.Timestamp
 |      business_type: str
 |          type of contract (see mappings.BSNTYPE)
 |      psr_type : str
 |          fil